# VI.9.32

Начальные данные

In [44]:
x_arr = [1910 + 10 * i for i in range(10)]
f_arr = [92228496, 106021537, 123202624, 132164569, 151325798, 179323175, 203211926, 226545805, 248709873, 281421906]

## Интерполянт в форме Ньютона

Вычисление разделенных разностей\
$f_N = [f_{0}, f_{01}, f_{012}, ..., f_{012...N-1}],$ где $N$ - количество узлов

In [45]:
f_N = [i for i in f_arr]
N = len(x_arr)
for i in range(1, N, 1):
    for j in range(N - 1, i - 1, -1):
        f_N[j] = (f_N[j] - f_N[j - 1]) / (x_arr[j] - x_arr[j - i])

Вычисление интерполянта в точке $x$

In [47]:
x = 2010
P_N = 0  # значение полинома в точке x
Poly_x = 1  # множитель перед разделенной разностью в полиноме
for i in range(N):
    if i > 0:
        Poly_x *= (x - x_arr[i - 1])
    P_N += f_N[i] * Poly_x
print(f"Экстраполированное значение численности населения в 2010 году: {P_N:.0f} человек")

Экстраполированное значение численности населения в 2010 году: 827906509 человек


Экстраполированное значение: 827 906 509\
Точное значение: 308 745 538